# Agent State From First Principles

**A Progressive Guide: From Single Agent to Multi-Agent Systems**

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** what agent state is conceptually (and what it is NOT)
2. **Design** state schemas before writing any code
3. **Observe** how state evolves across agent steps
4. **Validate** state correctness using evaluation functions
5. **Scale** state design from single agents to multi-agent systems

---

## Table of Contents

1. [What Is Agent State?](#section-1) — Conceptual foundations
2. [Stateless vs Stateful Agents](#section-2) — Why state matters
3. [Designing State Before Code](#section-3) — The critical first step
4. [Single-Agent, Multi-Step State](#section-4) — Practical implementation
5. [State Evaluation & Validation](#section-5) — Ensuring correctness
6. [State in a RAG Agent](#section-6) — Applied example
7. [Visual Diagrams of State Flow](#section-7) — Understanding through visualization
8. [From Single Agent to MAS](#section-8) — Scaling up
9. [Common State Design Mistakes](#section-9) — What to avoid
10. [Final Mental Model & Checklist](#section-10) — Key takeaways

---

## Setup: Required Libraries

Before we begin, let's import all the libraries we'll need throughout this notebook.

In [4]:
# Core Python libraries for type hints and data structures
from typing import TypedDict, Optional, Any, Callable
from dataclasses import dataclass, field, asdict
from datetime import datetime
from enum import Enum
import json
from pprint import pprint
from copy import deepcopy

# For validation and assertions
import sys
from typing import get_type_hints

# Display utilities
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print(f"Python version: {sys.version.split()[0]}")
print("-" * 50)

✅ All libraries imported successfully!
Python version: 3.13.0
--------------------------------------------------


### Helper Functions for State Display

Let's define some utility functions we'll use throughout the notebook to display and validate state.

In [5]:
def display_state(state: dict | object, title: str = "Current State") -> None:
    """
    Display agent state in a readable format.
    
    Args:
        state: Either a dict or dataclass instance
        title: Title to display above the state
    """
    print("=" * 60)
    print(f"  {title}")
    print("=" * 60)
    
    # Convert dataclass to dict if needed
    if hasattr(state, '__dataclass_fields__'):
        state_dict = asdict(state)
    else:
        state_dict = state
    
    # Pretty print the state
    for key, value in state_dict.items():
        if isinstance(value, list) and len(value) > 3:
            print(f"  {key}: [{len(value)} items]")
            for i, item in enumerate(value[:2], 1):
                print(f"    [{i}] {str(item)[:70]}...")
            print(f"    ... ({len(value) - 2} more items)")
        elif isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}...")
        else:
            print(f"  {key}: {value}")
    
    print("=" * 60)
    print()


def display_state_diff(old_state: dict, new_state: dict, step_name: str = "Step") -> None:
    """
    Display what changed between two states.
    
    Args:
        old_state: Previous state
        new_state: New state
        step_name: Name of the step that caused the change
    """
    print(f"\n🔄 Changes after: {step_name}")
    print("-" * 60)
    
    for key in new_state.keys():
        old_val = old_state.get(key)
        new_val = new_state.get(key)
        
        if old_val != new_val:
            print(f"  ✏️  {key}:")
            print(f"      Before: {old_val}")
            print(f"      After:  {new_val}")
    
    print("-" * 60)
    print()


def print_section_header(section_num: int, title: str) -> None:
    """Print a formatted section header."""
    print("\n" + "=" * 70)
    print(f"  SECTION {section_num}: {title.upper()}")
    print("=" * 70 + "\n")


# Test the helper functions
print("✅ Helper functions defined:")
print("   - display_state()")
print("   - display_state_diff()")
print("   - print_section_header()")
print("\nThese will help us visualize state changes throughout the notebook.")

✅ Helper functions defined:
   - display_state()
   - display_state_diff()
   - print_section_header()

These will help us visualize state changes throughout the notebook.


<a id="section-1"></a>
# Section 1: What Is Agent State?

## The Core Concept

**Agent state is the structured memory that an agent carries between reasoning steps.**

Think of it as the agent's "working memory" — everything it needs to remember to continue its task coherently.

---

## What Agent State IS

Agent state is:

| Characteristic | Description |
|----------------|-------------|
| **Explicit** | Clearly defined and inspectable at any point |
| **Structured** | Organized with named fields and types |
| **Mutable** | Changes as the agent progresses through steps |
| **Bounded** | Contains only what's necessary for the current task |
| **Observable** | Can be logged, validated, and debugged |

**Key Insight:** State is the *contract* between the agent's past and future steps. Without it, each step operates in isolation.

---

## What Agent State IS NOT

| Common Misconception | Reality |
|---------------------|----------|
| ❌ Just the conversation history | State includes structured data beyond messages |
| ❌ A Python class definition | The class is the *schema*; state is the *runtime instance* |
| ❌ Hidden inside the LLM | LLMs are stateless; we must manage state externally |
| ❌ The same as a database | State is transient working memory, not persistent storage |
| ❌ Optional for simple agents | Even "simple" agents have implicit state |

---

## Why LLMs Require Explicit State

LLMs are fundamentally **stateless functions**:

```
response = LLM(prompt)
```

Each call is independent. The LLM has no memory of:
- Previous calls
- What it decided earlier
- What tools it called
- What data it retrieved

### The Consequence

**Without explicit state, multi-step reasoning is impossible.**

Consider a research agent that must:
1. Search for papers
2. Read abstracts
3. Identify key themes
4. Write a summary

After step 2, how does the agent know which papers it found? The LLM doesn't remember — unless we explicitly store the results in state.

---

## Why "State ≠ Code Class"

This distinction is critical for beginners:

### The Schema (Code Definition)
```python
class AgentState(TypedDict):
    goal: str
    steps_completed: list[str]
    final_answer: str | None
```

This is a **blueprint** — it describes what state *could* look like.

### The State (Runtime Instance)
```python
current_state = {
    "goal": "Summarize recent AI papers",
    "steps_completed": ["search", "retrieve"],
    "final_answer": None
}
```

This is the **actual data** — the living memory of the agent.

### Why This Matters

- You can have a perfect schema but corrupted state
- You can have valid state that doesn't match your schema
- Debugging requires inspecting *state*, not just *schema*

---

## 🎯 Key Takeaway

> **Agent state is the explicit, structured, mutable memory that enables an LLM to reason across multiple steps. Without it, agents cannot exist — only isolated prompts.**

---

<a id="section-2"></a>
# Section 2: Stateless vs Stateful Agents

In this section, we'll demonstrate the difference between stateless and stateful approaches, and why message-only state quickly becomes insufficient.

---

## 2.1 Stateless Prompting

The simplest interaction with an LLM is stateless — each prompt is independent.

In [6]:
# Example: Stateless Prompting (Simulated)

def stateless_llm_call(prompt: str) -> str:
    """Simulates a stateless LLM call."""
    # In reality, this would call an LLM API
    return f"Response to: '{prompt[:50]}...'"

# Each call is completely independent
response1 = stateless_llm_call("What is machine learning?")
response2 = stateless_llm_call("Can you explain more about that?")  # "that" has no referent!

print("Response 1:", response1)
print("Response 2:", response2)
print()
print("⚠️ Problem: The LLM has no idea what 'that' refers to!")

Response 1: Response to: 'What is machine learning?...'
Response 2: Response to: 'Can you explain more about that?...'

⚠️ Problem: The LLM has no idea what 'that' refers to!


### The Stateless Problem

In stateless prompting:
- Each call exists in isolation
- No memory of previous interactions
- Context must be fully re-supplied every time
- Multi-turn conversations are impossible

---

## 2.2 Message-Only State (SimpleState)

The most common first step toward statefulness is **message history** — storing the conversation.

In [7]:
from typing import TypedDict

class SimpleAgentState(TypedDict):
    """The most basic form of agent state: just messages."""
    messages: list[dict]  # [{"role": "user", "content": "..."}, ...]

# Initialize state
simple_state: SimpleAgentState = {
    "messages": []
}

def add_message(state: SimpleAgentState, role: str, content: str) -> SimpleAgentState:
    """Add a message to state."""
    state["messages"].append({"role": role, "content": content})
    return state

def display_state(state: SimpleAgentState) -> None:
    """Display current state."""
    print("=" * 50)
    print("CURRENT STATE")
    print("=" * 50)
    for i, msg in enumerate(state["messages"], 1):
        print(f"{i}. [{msg['role'].upper()}]: {msg['content'][:60]}..." 
              if len(msg['content']) > 60 else f"{i}. [{msg['role'].upper()}]: {msg['content']}")
    print("=" * 50)

# Simulate a conversation
simple_state = add_message(simple_state, "user", "What is machine learning?")
simple_state = add_message(simple_state, "assistant", "Machine learning is a subset of AI...")
simple_state = add_message(simple_state, "user", "Can you explain more about that?")

display_state(simple_state)
print()
print("✅ Now the agent can see the full conversation history!")

CURRENT STATE
1. [USER]: What is machine learning?
2. [ASSISTANT]: Machine learning is a subset of AI...
3. [USER]: Can you explain more about that?

✅ Now the agent can see the full conversation history!


### SimpleState Works For...

- Basic chatbots
- Single-turn Q&A
- Simple conversational agents

---

## 2.3 Why Message-Only State Fails for Complex Agents

Consider an agent that must:
1. **Search** for documents
2. **Retrieve** relevant content
3. **Analyze** the content
4. **Generate** a response

With only message history, we face critical problems:

In [8]:
# Demonstrating the limitations of SimpleState

# Simulated agent workflow with message-only state
complex_state: SimpleAgentState = {"messages": []}

# Step 1: User query
complex_state = add_message(complex_state, "user", "Find papers about transformer architectures")

# Step 2: Agent searches (but where do we store the results?)
search_results = ["Paper A: Attention Is All You Need", "Paper B: BERT", "Paper C: GPT-3"]
complex_state = add_message(complex_state, "assistant", f"Found papers: {search_results}")

# Step 3: Agent retrieves content (results are buried in message text!)
retrieved_content = {"Paper A": "The Transformer architecture...", "Paper B": "BERT uses..."}
complex_state = add_message(complex_state, "assistant", f"Retrieved: {retrieved_content}")

display_state(complex_state)

print()
print("❌ PROBLEMS WITH THIS APPROACH:")
print("-" * 40)
print("1. Search results are buried in message strings")
print("2. No structured access to retrieved documents")
print("3. Cannot validate if retrieval happened")
print("4. Difficult to debug or inspect intermediate results")
print("5. State is implicit, not explicit")

CURRENT STATE
1. [USER]: Find papers about transformer architectures
2. [ASSISTANT]: Found papers: ['Paper A: Attention Is All You Need', 'Paper ...
3. [ASSISTANT]: Retrieved: {'Paper A': 'The Transformer architecture...', 'P...

❌ PROBLEMS WITH THIS APPROACH:
----------------------------------------
1. Search results are buried in message strings
2. No structured access to retrieved documents
3. Cannot validate if retrieval happened
4. Difficult to debug or inspect intermediate results
5. State is implicit, not explicit


### The Core Limitation

**Message-only state conflates communication with computation.**

| What Messages Are For | What State Is For |
|----------------------|-------------------|
| Human-agent communication | Agent's working memory |
| Natural language | Structured data |
| Linear conversation | Branching computation |
| Display to users | Internal processing |

When you try to store structured data (search results, retrieved documents, analysis outputs) in messages:

- You lose type safety
- You cannot validate state
- You cannot easily inspect or debug
- You create fragile, unparseable strings

---

## 🎯 Key Takeaway

> **SimpleState (message-only) is sufficient for chatbots but fails for agents that perform multi-step computation. Complex agents need structured state with explicit fields for each type of data they work with.**

---

<a id="section-3"></a>
# Section 3: Designing State Before Code

This is the most critical section of this notebook. **State design must come before implementation.**

---

## 3.1 The First Question to Ask

Before writing any agent code, ask:

> **"What must this agent remember between steps?"**

This question forces you to think about:
- What information flows between steps
- What intermediate results must be preserved
- What decisions must be traceable

---

## 3.2 Enumerating Conceptual Memory

For any agent task, enumerate the conceptual categories of memory:

### Example: Research Agent

| Memory Category | Purpose | Example Content |
|----------------|---------|------------------|
| **Goal** | What is the agent trying to achieve? | "Summarize recent papers on LLMs" |
| **Progress** | What steps have been completed? | ["search", "retrieve", "analyze"] |
| **Inputs** | What data was provided or retrieved? | List of paper titles, abstracts |
| **Intermediate** | What has been computed but not finalized? | Draft summary, key themes |
| **Decisions** | What choices has the agent made? | Selected 3 of 10 papers |
| **Output** | What is the final deliverable? | Final summary text |

---

## 3.3 Why State Defines the Agent's Reasoning Boundary

**An agent can only reason about what exists in its state.**

If the state doesn't include:
- Retrieved documents → Agent cannot cite them
- Previous decisions → Agent cannot explain its reasoning
- Error history → Agent cannot recover from failures
- Progress markers → Agent cannot know where it is

### The Boundary Principle

```
┌─────────────────────────────────────┐
│           AGENT'S WORLD             │
│                                     │
│   Everything in state is visible    │
│   Everything outside is invisible   │
│                                     │
│   State = Reasoning Boundary        │
└─────────────────────────────────────┘
```

---

## 3.4 State Design Checklist

Use this checklist before implementing any agent:

### ☑️ State Design Checklist

| # | Question | Check |
|---|----------|-------|
| 1 | Have I defined the agent's **goal** field? | ☐ |
| 2 | Can I track **what steps have been completed**? | ☐ |
| 3 | Do I store **all inputs** the agent receives? | ☐ |
| 4 | Do I store **all intermediate results**? | ☐ |
| 5 | Can I trace **decisions** the agent made? | ☐ |
| 6 | Is the **final output** explicitly stored? | ☐ |
| 7 | Can I **validate** the state at any step? | ☐ |
| 8 | Can I **serialize** the state (for logging/debugging)? | ☐ |
| 9 | Is every field **necessary** (no bloat)? | ☐ |
| 10 | Is the schema **typed** (TypedDict, dataclass, Pydantic)? | ☐ |

---

## 3.5 Conceptual Schema: Non-Code Design

Before writing any Python, design your state on paper (or in markdown).

### Example: Research Agent Conceptual Schema

```
RESEARCH AGENT STATE
====================

1. GOAL
   - Type: String
   - Purpose: The research question or task
   - Example: "What are the key advances in LLMs in 2024?"
   - Required: Yes

2. SEARCH QUERY
   - Type: String
   - Purpose: The query sent to the search system
   - Example: "LLM advances 2024 transformer"
   - Required: Yes (after planning step)

3. SEARCH RESULTS
   - Type: List of {title, url, snippet}
   - Purpose: Raw search results before filtering
   - Required: Yes (after search step)

4. SELECTED DOCUMENTS
   - Type: List of {title, content, relevance_score}
   - Purpose: Documents chosen for analysis
   - Required: Yes (after retrieval step)

5. KEY FINDINGS
   - Type: List of Strings
   - Purpose: Main points extracted from documents
   - Required: Yes (after analysis step)

6. DRAFT SUMMARY
   - Type: String
   - Purpose: Initial summary before refinement
   - Required: Yes (after synthesis step)

7. FINAL ANSWER
   - Type: String
   - Purpose: The deliverable to return to user
   - Required: Yes (at completion)

8. STEPS COMPLETED
   - Type: List of Strings
   - Purpose: Track progress through workflow
   - Example: ["plan", "search", "retrieve", "analyze"]
   - Required: Yes (always)

9. ERRORS
   - Type: List of {step, message, timestamp}
   - Purpose: Track any errors for debugging
   - Required: No (only if errors occur)
```

---

## 3.6 From Conceptual Schema to Code Schema

Once you have the conceptual design, translation to code is straightforward:

In [9]:
from typing import TypedDict, Optional
from dataclasses import dataclass, field
from datetime import datetime

# Option 1: TypedDict (simple, good for JSON-like structures)
class SearchResult(TypedDict):
    title: str
    url: str
    snippet: str

class Document(TypedDict):
    title: str
    content: str
    relevance_score: float

class ErrorRecord(TypedDict):
    step: str
    message: str
    timestamp: str

class ResearchAgentState(TypedDict):
    """Complete state schema for a research agent."""
    # Core
    goal: str
    steps_completed: list[str]
    
    # Search phase
    search_query: Optional[str]
    search_results: list[SearchResult]
    
    # Retrieval phase
    selected_documents: list[Document]
    
    # Analysis phase
    key_findings: list[str]
    draft_summary: Optional[str]
    
    # Output
    final_answer: Optional[str]
    
    # Meta
    errors: list[ErrorRecord]

# Display the schema structure
print("ResearchAgentState Schema")
print("=" * 40)
for field_name, field_type in ResearchAgentState.__annotations__.items():
    print(f"  {field_name}: {field_type}")

ResearchAgentState Schema
  goal: <class 'str'>
  steps_completed: list[str]
  search_query: typing.Optional[str]
  search_results: list[__main__.SearchResult]
  selected_documents: list[__main__.Document]
  key_findings: list[str]
  draft_summary: typing.Optional[str]
  final_answer: typing.Optional[str]
  errors: list[__main__.ErrorRecord]


In [10]:
# Option 2: Dataclass (more Pythonic, with defaults and methods)
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class ResearchAgentStateV2:
    """Research agent state using dataclass."""
    # Required fields
    goal: str
    
    # Fields with defaults
    steps_completed: list[str] = field(default_factory=list)
    search_query: Optional[str] = None
    search_results: list[dict] = field(default_factory=list)
    selected_documents: list[dict] = field(default_factory=list)
    key_findings: list[str] = field(default_factory=list)
    draft_summary: Optional[str] = None
    final_answer: Optional[str] = None
    errors: list[dict] = field(default_factory=list)
    
    def is_complete(self) -> bool:
        """Check if agent has finished."""
        return self.final_answer is not None
    
    def current_step(self) -> str:
        """Get the last completed step."""
        return self.steps_completed[-1] if self.steps_completed else "not_started"

# Create and display an instance
state = ResearchAgentStateV2(goal="Summarize recent LLM papers")
print("Initial State:")
print(state)
print()
print(f"Is complete? {state.is_complete()}")
print(f"Current step: {state.current_step()}")

Initial State:
ResearchAgentStateV2(goal='Summarize recent LLM papers', steps_completed=[], search_query=None, search_results=[], selected_documents=[], key_findings=[], draft_summary=None, final_answer=None, errors=[])

Is complete? False
Current step: not_started


---

## 3.7 The Design-First Workflow

Always follow this order:

```
1. UNDERSTAND THE TASK
   └── What is the agent supposed to do?
   
2. ENUMERATE STEPS
   └── What steps must the agent take?
   
3. IDENTIFY STATE FIELDS
   └── What must be remembered between steps?
   
4. DESIGN CONCEPTUAL SCHEMA
   └── Write it down in plain language
   
5. TRANSLATE TO CODE SCHEMA
   └── TypedDict, dataclass, or Pydantic
   
6. IMPLEMENT AGENT LOGIC
   └── Now you can write the agent code
```

**Notice:** Code comes LAST, not first.

---

## 🎯 Key Takeaway

> **State design is the most critical step in building an agent. If you get the state wrong, no amount of clever prompting or tool calling will fix it. Design the state schema completely before writing any agent code.**

---

---

# Summary: Sections 1-3

| Section | Key Concept |
|---------|-------------|
| **Section 1** | Agent state is explicit, structured memory that enables multi-step reasoning |
| **Section 2** | Message-only state fails for complex agents; structured state is required |
| **Section 3** | Design state schemas BEFORE writing code; state defines the reasoning boundary |

---

## Next Sections Preview

In the upcoming sections, we will:

- **Section 4:** Implement a single-agent, multi-step system with state evolution
- **Section 5:** Add validation and evaluation to ensure state correctness
- **Section 6:** Build a RAG agent with proper state management
- **Section 7:** Visualize state flow with diagrams
- **Section 8:** Scale to multi-agent systems
- **Section 9:** Learn common state design mistakes
- **Section 10:** Consolidate with a final mental model

---

<a id="section-4"></a>
# Section 4: Single-Agent, Multi-Step State

Now that we understand how to design state schemas, let's implement a practical single-agent system that performs multiple steps.

We'll build a **Task Planning Agent** that:
1. Receives a complex task
2. Breaks it into subtasks
3. Executes each subtask
4. Produces a final result

---

## 4.1 Defining the State Schema

First, let's design the state schema with explicit fields for each phase.

In [11]:
@dataclass
class TaskPlannerState:
    """
    State for a task planning agent.
    
    Each field has a specific purpose in the agent's workflow.
    """
    # INPUT: What the agent is trying to accomplish
    goal: str
    
    # TRACKING: Which steps have been completed
    steps_completed: list[str] = field(default_factory=list)
    
    # INTERMEDIATE: The plan created by the agent
    subtasks: list[str] = field(default_factory=list)
    
    # INTERMEDIATE: Results from executing each subtask
    intermediate_results: dict[str, str] = field(default_factory=dict)
    
    # OUTPUT: The final deliverable
    final_output: Optional[str] = None
    
    # META: Timestamp of creation
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())


# Why each field exists:
print("📋 STATE FIELD RATIONALE")
print("=" * 60)
print()
print("1. goal: str")
print("   → WHY: Agent needs to remember what it's trying to achieve")
print()
print("2. steps_completed: list[str]")
print("   → WHY: Tracks progress; prevents duplicate work")
print()
print("3. subtasks: list[str]")
print("   → WHY: Stores the plan; enables validation and debugging")
print()
print("4. intermediate_results: dict[str, str]")
print("   → WHY: Preserves outputs from each subtask")
print()
print("5. final_output: Optional[str]")
print("   → WHY: Marks completion; contains final deliverable")
print()
print("6. created_at: str")
print("   → WHY: Enables timeline tracking and debugging")
print()
print("=" * 60)

📋 STATE FIELD RATIONALE

1. goal: str
   → WHY: Agent needs to remember what it's trying to achieve

2. steps_completed: list[str]
   → WHY: Tracks progress; prevents duplicate work

3. subtasks: list[str]
   → WHY: Stores the plan; enables validation and debugging

4. intermediate_results: dict[str, str]
   → WHY: Preserves outputs from each subtask

5. final_output: Optional[str]
   → WHY: Marks completion; contains final deliverable

6. created_at: str
   → WHY: Enables timeline tracking and debugging



## 4.2 Implementing the Multi-Step Agent

Now let's implement the agent functions that modify state at each step.

In [12]:
def step1_plan(state: TaskPlannerState) -> TaskPlannerState:
    """
    Step 1: Break the goal into subtasks.
    
    This simulates an LLM planning step.
    """
    print("🔵 STEP 1: Planning")
    print("-" * 40)
    
    # Simulate planning (in reality, this would call an LLM)
    if "organize a conference" in state.goal.lower():
        state.subtasks = [
            "Find venue",
            "Create agenda",
            "Invite speakers",
            "Setup registration"
        ]
    else:
        # Default planning for other tasks
        state.subtasks = [
            "Research requirements",
            "Create outline",
            "Execute plan"
        ]
    
    state.steps_completed.append("plan")
    
    print(f"Goal: {state.goal}")
    print(f"Generated {len(state.subtasks)} subtasks:")
    for i, task in enumerate(state.subtasks, 1):
        print(f"  {i}. {task}")
    print("-" * 40)
    
    return state


def step2_execute(state: TaskPlannerState) -> TaskPlannerState:
    """
    Step 2: Execute each subtask and store results.
    
    This simulates task execution.
    """
    print("\n🔵 STEP 2: Executing Subtasks")
    print("-" * 40)
    
    for i, subtask in enumerate(state.subtasks, 1):
        # Simulate execution (in reality, this might call tools or LLMs)
        result = f"Completed: {subtask} (simulated result)"
        state.intermediate_results[subtask] = result
        print(f"  ✓ Subtask {i}: {subtask}")
        print(f"    Result: {result}")
    
    state.steps_completed.append("execute")
    print("-" * 40)
    
    return state


def step3_synthesize(state: TaskPlannerState) -> TaskPlannerState:
    """
    Step 3: Combine intermediate results into final output.
    
    This simulates synthesis/summarization.
    """
    print("\n🔵 STEP 3: Synthesizing Final Output")
    print("-" * 40)
    
    # Combine all results
    summary_parts = [f"Goal: {state.goal}", ""]
    summary_parts.append("Completed Tasks:")
    
    for i, (task, result) in enumerate(state.intermediate_results.items(), 1):
        summary_parts.append(f"{i}. {task}")
        summary_parts.append(f"   → {result}")
    
    summary_parts.append("")
    summary_parts.append(f"All {len(state.subtasks)} subtasks completed successfully.")
    
    state.final_output = "\n".join(summary_parts)
    state.steps_completed.append("synthesize")
    
    print("Final output generated!")
    print("-" * 40)
    
    return state


print("✅ Agent functions defined:")
print("   - step1_plan()")
print("   - step2_execute()")
print("   - step3_synthesize()")

✅ Agent functions defined:
   - step1_plan()
   - step2_execute()
   - step3_synthesize()


## 4.3 Running the Agent: Observing State Evolution

Let's run the agent and **observe how state changes at each step**.

In [15]:
# Initialize state
state = TaskPlannerState(goal="Organize a conference on AI Safety")

print("=" * 70)
print("  AGENT EXECUTION: STATE EVOLUTION")
print("=" * 70)
print()

# SNAPSHOT 0: Initial state
print("📸 SNAPSHOT 0: Initial State")
print("=" * 60)
for key, value in asdict(state).items():
    if isinstance(value, str) and len(value) > 100:
        print(f"  {key}: {value[:100]}...")
    else:
        print(f"  {key}: {value}")
print("=" * 60)
print()

# STEP 1: Planning
state = step1_plan(state)
print("\n📸 SNAPSHOT 1: After Planning")
print("=" * 60)
for key, value in asdict(state).items():
    if isinstance(value, str) and len(value) > 100:
        print(f"  {key}: {value[:100]}...")
    else:
        print(f"  {key}: {value}")
print("=" * 60)
print()

# STEP 2: Execution
state = step2_execute(state)
print("\n📸 SNAPSHOT 2: After Execution")
print("=" * 60)
for key, value in asdict(state).items():
    if isinstance(value, str) and len(value) > 100:
        print(f"  {key}: {value[:100]}...")
    else:
        print(f"  {key}: {value}")
print("=" * 60)
print()

# STEP 3: Synthesis
state = step3_synthesize(state)
print("\n📸 SNAPSHOT 3: After Synthesis")
print("=" * 60)
for key, value in asdict(state).items():
    if isinstance(value, str) and len(value) > 100:
        print(f"  {key}: {value[:100]}...")
    else:
        print(f"  {key}: {value}")
print("=" * 60)
print()

# Display the final output
print("\n" + "=" * 70)
print("  FINAL DELIVERABLE")
print("=" * 70)
print(state.final_output)
print("=" * 70)

  AGENT EXECUTION: STATE EVOLUTION

📸 SNAPSHOT 0: Initial State
  goal: Organize a conference on AI Safety
  steps_completed: []
  subtasks: []
  intermediate_results: {}
  final_output: None
  created_at: 2025-12-22T18:08:55.636488

🔵 STEP 1: Planning
----------------------------------------
Goal: Organize a conference on AI Safety
Generated 4 subtasks:
  1. Find venue
  2. Create agenda
  3. Invite speakers
  4. Setup registration
----------------------------------------

📸 SNAPSHOT 1: After Planning
  goal: Organize a conference on AI Safety
  steps_completed: ['plan']
  subtasks: ['Find venue', 'Create agenda', 'Invite speakers', 'Setup registration']
  intermediate_results: {}
  final_output: None
  created_at: 2025-12-22T18:08:55.636488


🔵 STEP 2: Executing Subtasks
----------------------------------------
  ✓ Subtask 1: Find venue
    Result: Completed: Find venue (simulated result)
  ✓ Subtask 2: Create agenda
    Result: Completed: Create agenda (simulated result)
  ✓ Subtask

## 4.4 Key Observations

### What We Learned

1. **State is Explicit**: At any point, we can inspect what the agent knows
2. **State Evolves**: Each step adds new information without losing previous data
3. **State is Structured**: We can access specific fields programmatically
4. **State Enables Debugging**: We can see exactly where the agent is in its workflow

### The Multi-Step Pattern

```
Initialize State
    ↓
Step 1: Modify State
    ↓
Step 2: Modify State  
    ↓
Step 3: Modify State
    ↓
Final State (contains complete history)
```

---

## 🎯 Key Takeaway

> **Multi-step agents require state that accumulates information across steps. Each step reads from state and writes to state, creating a traceable chain of reasoning.**

---

<a id="section-5"></a>
# Section 5: State Evaluation & Validation

Having state is not enough — we must **validate** that the state is correct at each step.

Without validation:
- Bugs hide in state
- Agents fail silently
- Debugging becomes impossible

---

## 5.1 Why State Validation Matters

**State validation answers critical questions:**

| Question | Why It Matters |
|----------|----------------|
| Are required fields present? | Agent cannot proceed without necessary data |
| Are fields the correct type? | Type errors cause runtime failures |
| Is data in the expected format? | Malformed data breaks downstream steps |
| Did steps execute in order? | Some steps depend on previous steps |
| Are invariants maintained? | Business logic constraints must hold |

**Validation is the difference between a production agent and a demo.**

---

## 5.2 Building Validation Functions

In [16]:
class StateValidationError(Exception):
    """Raised when state validation fails."""
    pass


def validate_task_planner_state(state: TaskPlannerState, expected_step: str) -> bool:
    """
    Validate TaskPlannerState at a specific step.
    
    Args:
        state: The state to validate
        expected_step: The step we're validating for
    
    Returns:
        True if valid
    
    Raises:
        StateValidationError if validation fails
    """
    errors = []
    
    # RULE 1: Goal must always be present
    if not state.goal or state.goal.strip() == "":
        errors.append("Goal is missing or empty")
    
    # RULE 2: Validate step-specific requirements
    if expected_step == "plan":
        if "plan" not in state.steps_completed:
            errors.append("Planning step not marked as completed")
        if len(state.subtasks) == 0:
            errors.append("Planning step completed but no subtasks created")
    
    elif expected_step == "execute":
        # Must have planned first
        if "plan" not in state.steps_completed:
            errors.append("Cannot execute before planning")
        if "execute" not in state.steps_completed:
            errors.append("Execution step not marked as completed")
        # Must have results for all subtasks
        if len(state.intermediate_results) != len(state.subtasks):
            errors.append(
                f"Expected {len(state.subtasks)} results, "
                f"got {len(state.intermediate_results)}"
            )
    
    elif expected_step == "synthesize":
        # Must have executed first
        if "execute" not in state.steps_completed:
            errors.append("Cannot synthesize before execution")
        if state.final_output is None or state.final_output.strip() == "":
            errors.append("Synthesis step completed but final_output is empty")
    
    # If there are errors, raise exception
    if errors:
        error_msg = "\n".join([f"  ❌ {e}" for e in errors])
        raise StateValidationError(f"State validation failed:\n{error_msg}")
    
    print(f"✅ State validation passed for step: {expected_step}")
    return True


# Create helper function for safe validation
def safe_validate(state: TaskPlannerState, step: str) -> None:
    """Validate and print result."""
    try:
        validate_task_planner_state(state, step)
    except StateValidationError as e:
        print(f"🔴 VALIDATION FAILED")
        print(str(e))


print("✅ Validation functions defined:")
print("   - validate_task_planner_state()")
print("   - safe_validate()")

✅ Validation functions defined:
   - validate_task_planner_state()
   - safe_validate()


## 5.3 Demonstrating Validation: Correct Case

Let's validate a properly executed agent workflow.

In [17]:
print("=" * 70)
print("  CORRECT WORKFLOW WITH VALIDATION")
print("=" * 70)
print()

# Run agent with validation at each step
correct_state = TaskPlannerState(goal="Plan a team retreat")

# Step 1: Plan and validate
correct_state = step1_plan(correct_state)
safe_validate(correct_state, "plan")
print()

# Step 2: Execute and validate
correct_state = step2_execute(correct_state)
safe_validate(correct_state, "execute")
print()

# Step 3: Synthesize and validate
correct_state = step3_synthesize(correct_state)
safe_validate(correct_state, "synthesize")

print()
print("=" * 70)
print("✅ All validation checks passed! Agent executed correctly.")
print("=" * 70)

  CORRECT WORKFLOW WITH VALIDATION

🔵 STEP 1: Planning
----------------------------------------
Goal: Plan a team retreat
Generated 3 subtasks:
  1. Research requirements
  2. Create outline
  3. Execute plan
----------------------------------------
✅ State validation passed for step: plan


🔵 STEP 2: Executing Subtasks
----------------------------------------
  ✓ Subtask 1: Research requirements
    Result: Completed: Research requirements (simulated result)
  ✓ Subtask 2: Create outline
    Result: Completed: Create outline (simulated result)
  ✓ Subtask 3: Execute plan
    Result: Completed: Execute plan (simulated result)
----------------------------------------
✅ State validation passed for step: execute


🔵 STEP 3: Synthesizing Final Output
----------------------------------------
Final output generated!
----------------------------------------
✅ State validation passed for step: synthesize

✅ All validation checks passed! Agent executed correctly.


## 5.4 Demonstrating Validation: Failure Cases

Now let's see what happens when state is malformed or steps are executed out of order.

In [18]:
print("=" * 70)
print("  FAILURE CASE 1: Missing Required Step")
print("=" * 70)
print()

# Create state and try to execute WITHOUT planning
bad_state_1 = TaskPlannerState(goal="Build a website")
bad_state_1.steps_completed = ["execute"]  # Claim execution happened
bad_state_1.intermediate_results = {"fake": "result"}

print("Attempting to validate execution step WITHOUT planning first:")
safe_validate(bad_state_1, "execute")

print("\n" + "=" * 70)
print("  FAILURE CASE 2: Empty Goal")
print("=" * 70)
print()

# Create state with empty goal
bad_state_2 = TaskPlannerState(goal="")
bad_state_2 = step1_plan(bad_state_2)

print("Attempting to validate with empty goal:")
safe_validate(bad_state_2, "plan")

print("\n" + "=" * 70)
print("  FAILURE CASE 3: Incomplete Execution")
print("=" * 70)
print()

# Create state where execution didn't process all subtasks
bad_state_3 = TaskPlannerState(goal="Create marketing campaign")
bad_state_3.subtasks = ["Research", "Design", "Launch"]
bad_state_3.steps_completed = ["plan", "execute"]
bad_state_3.intermediate_results = {"Research": "Done"}  # Only 1 of 3!

print("Attempting to validate with incomplete execution:")
safe_validate(bad_state_3, "execute")

print("\n" + "=" * 70)
print("🎓 LESSON: Validation catches errors that would otherwise fail silently!")
print("=" * 70)

  FAILURE CASE 1: Missing Required Step

Attempting to validate execution step WITHOUT planning first:
🔴 VALIDATION FAILED
State validation failed:
  ❌ Cannot execute before planning
  ❌ Expected 0 results, got 1

  FAILURE CASE 2: Empty Goal

🔵 STEP 1: Planning
----------------------------------------
Goal: 
Generated 3 subtasks:
  1. Research requirements
  2. Create outline
  3. Execute plan
----------------------------------------
Attempting to validate with empty goal:
🔴 VALIDATION FAILED
State validation failed:
  ❌ Goal is missing or empty

  FAILURE CASE 3: Incomplete Execution

Attempting to validate with incomplete execution:
🔴 VALIDATION FAILED
State validation failed:
  ❌ Expected 3 results, got 1

🎓 LESSON: Validation catches errors that would otherwise fail silently!


## 5.5 State Validation Best Practices

### When to Validate

✅ **After each agent step**  
✅ **Before critical operations**  
✅ **When deserializing state from storage**  
✅ **In production monitoring**

### What to Validate

| Category | Examples |
|----------|----------|
| **Presence** | Required fields exist and are not None |
| **Type** | Fields match expected types |
| **Format** | Dates, URLs, IDs follow correct format |
| **Order** | Steps executed in correct sequence |
| **Invariants** | Business rules hold (e.g., sum = 100%) |
| **Completeness** | All expected data present |

---

## 🎯 Key Takeaway

> **State validation is not optional. In production agents, every state transition must be validated. Validation converts silent failures into explicit, debuggable errors.**

---

<a id="section-6"></a>
# Section 6: State in a RAG Agent

Let's apply everything we've learned to a **Retrieval-Augmented Generation (RAG)** agent.

RAG agents are particularly interesting because they have distinct phases:
1. **Query formulation**
2. **Document retrieval**
3. **Chunk selection**
4. **Answer generation**

Each phase produces outputs that must be stored in state.

---

## 6.1 Why Retrieval Outputs Belong in State

### The Problem Without State

If retrieval results are **not** in state:
- Agent cannot cite sources
- We cannot debug what was retrieved
- We cannot validate retrieval quality
- We cannot trace reasoning from docs → answer

### The Solution: Explicit Retrieval State

Store:
- What was retrieved
- What was selected
- Why it was selected
- How it was used

---

## 6.2 Designing the RAG State Schema

In [19]:
@dataclass
class Document:
    """A retrieved document."""
    id: str
    title: str
    content: str
    relevance_score: float = 0.0


@dataclass
class RAGAgentState:
    """
    State for a Retrieval-Augmented Generation agent.
    
    Each field represents a distinct phase of the RAG workflow.
    """
    # INPUT: User's question
    query: str
    
    # RETRIEVAL PHASE: Raw documents from search
    retrieved_documents: list[Document] = field(default_factory=list)
    
    # SELECTION PHASE: Filtered/ranked chunks
    selected_chunks: list[str] = field(default_factory=list)
    
    # GENERATION PHASE: Draft answer
    draft_answer: Optional[str] = None
    
    # OUTPUT: Final answer with citations
    final_answer: Optional[str] = None
    
    # META: Track workflow steps
    steps_completed: list[str] = field(default_factory=list)


print("✅ RAG State Schema Defined")
print()
print("State Fields:")
print("  1. query              → User's question")
print("  2. retrieved_documents → Raw search results")
print("  3. selected_chunks     → Filtered relevant content")
print("  4. draft_answer        → Initial generated answer")
print("  5. final_answer        → Polished answer with citations")
print("  6. steps_completed     → Workflow tracking")

✅ RAG State Schema Defined

State Fields:
  1. query              → User's question
  2. retrieved_documents → Raw search results
  3. selected_chunks     → Filtered relevant content
  4. draft_answer        → Initial generated answer
  5. final_answer        → Polished answer with citations
  6. steps_completed     → Workflow tracking


## 6.3 Implementing the RAG Workflow

In [20]:
def rag_step1_retrieve(state: RAGAgentState) -> RAGAgentState:
    """Step 1: Retrieve documents based on query."""
    print("🔍 STEP 1: Retrieving Documents")
    print("-" * 50)
    
    # Simulate document retrieval (would use vector DB in production)
    mock_docs = [
        Document(
            id="doc1",
            title="Introduction to Transformers",
            content="Transformers use self-attention mechanisms to process sequences...",
            relevance_score=0.92
        ),
        Document(
            id="doc2",
            title="BERT: Bidirectional Encoder Representations",
            content="BERT is a transformer-based model that learns bidirectional representations...",
            relevance_score=0.87
        ),
        Document(
            id="doc3",
            title="GPT Architecture Overview",
            content="GPT uses a decoder-only transformer architecture for autoregressive generation...",
            relevance_score=0.85
        ),
        Document(
            id="doc4",
            title="History of Neural Networks",
            content="Neural networks date back to the 1950s with the perceptron...",
            relevance_score=0.34
        ),
    ]
    
    state.retrieved_documents = mock_docs
    state.steps_completed.append("retrieve")
    
    print(f"Query: '{state.query}'")
    print(f"Retrieved {len(mock_docs)} documents:")
    for doc in mock_docs:
        print(f"  • {doc.title} (score: {doc.relevance_score:.2f})")
    print("-" * 50)
    
    return state


def rag_step2_select(state: RAGAgentState) -> RAGAgentState:
    """Step 2: Select relevant chunks from retrieved documents."""
    print("\n📑 STEP 2: Selecting Relevant Chunks")
    print("-" * 50)
    
    # Filter documents by relevance threshold
    threshold = 0.8
    relevant_docs = [d for d in state.retrieved_documents if d.relevance_score >= threshold]
    
    # Extract content chunks
    state.selected_chunks = [doc.content for doc in relevant_docs]
    state.steps_completed.append("select")
    
    print(f"Relevance threshold: {threshold}")
    print(f"Selected {len(state.selected_chunks)} chunks from {len(relevant_docs)} docs")
    for i, chunk in enumerate(state.selected_chunks, 1):
        print(f"  {i}. {chunk[:60]}...")
    print("-" * 50)
    
    return state


def rag_step3_generate(state: RAGAgentState) -> RAGAgentState:
    """Step 3: Generate answer using selected chunks."""
    print("\n✍️  STEP 3: Generating Answer")
    print("-" * 50)
    
    # Simulate LLM generation (would call actual LLM in production)
    context = "\n\n".join(state.selected_chunks)
    
    state.draft_answer = (
        f"Based on the retrieved information:\n\n"
        f"{state.selected_chunks[0][:100]}...\n\n"
        f"Transformers represent a significant advancement in neural architecture."
    )
    
    # Add citations
    cited_docs = [d.title for d in state.retrieved_documents 
                  if d.relevance_score >= 0.8]
    
    state.final_answer = (
        f"{state.draft_answer}\n\n"
        f"Sources: {', '.join(cited_docs)}"
    )
    
    state.steps_completed.append("generate")
    
    print("Draft answer generated")
    print("Citations added")
    print("-" * 50)
    
    return state


print("✅ RAG workflow functions defined:")
print("   - rag_step1_retrieve()")
print("   - rag_step2_select()")
print("   - rag_step3_generate()")

✅ RAG workflow functions defined:
   - rag_step1_retrieve()
   - rag_step2_select()
   - rag_step3_generate()


## 6.4 Running the RAG Agent

In [22]:
# Initialize RAG agent
rag_state = RAGAgentState(query="What are transformers in machine learning?")

print("=" * 70)
print("  RAG AGENT EXECUTION")
print("=" * 70)
print()

# Execute workflow
rag_state = rag_step1_retrieve(rag_state)
rag_state = rag_step2_select(rag_state)
rag_state = rag_step3_generate(rag_state)

# Display final state
print("\n" + "=" * 70)
print("  FINAL RAG STATE")
print("=" * 70)
for key, value in asdict(rag_state).items():
    if isinstance(value, list) and len(value) > 3:
        print(f"  {key}: [{len(value)} items]")
    elif isinstance(value, str) and len(value) > 100:
        print(f"  {key}: {value[:100]}...")
    else:
        print(f"  {key}: {value}")
print("=" * 70)
print()

# Display the answer
print("=" * 70)
print("  GENERATED ANSWER")
print("=" * 70)
print(rag_state.final_answer)
print("=" * 70)

  RAG AGENT EXECUTION

🔍 STEP 1: Retrieving Documents
--------------------------------------------------
Query: 'What are transformers in machine learning?'
Retrieved 4 documents:
  • Introduction to Transformers (score: 0.92)
  • BERT: Bidirectional Encoder Representations (score: 0.87)
  • GPT Architecture Overview (score: 0.85)
  • History of Neural Networks (score: 0.34)
--------------------------------------------------

📑 STEP 2: Selecting Relevant Chunks
--------------------------------------------------
Relevance threshold: 0.8
Selected 3 chunks from 3 docs
  1. Transformers use self-attention mechanisms to process sequen...
  2. BERT is a transformer-based model that learns bidirectional ...
  3. GPT uses a decoder-only transformer architecture for autoreg...
--------------------------------------------------

✍️  STEP 3: Generating Answer
--------------------------------------------------
Draft answer generated
Citations added
-------------------------------------------------

## 6.5 Validating RAG State

Let's create validation to ensure retrieval happens before generation.

In [23]:
def validate_rag_state(state: RAGAgentState) -> bool:
    """
    Validate RAG agent state with specific invariants.
    
    Key invariants:
    1. Retrieval must occur before selection
    2. Selection must occur before generation
    3. Generated answers must reference retrieved documents
    """
    errors = []
    
    # Check query exists
    if not state.query or state.query.strip() == "":
        errors.append("Query is missing or empty")
    
    # Validate retrieval
    if "retrieve" in state.steps_completed:
        if len(state.retrieved_documents) == 0:
            errors.append("Retrieval step completed but no documents retrieved")
    
    # Validate selection
    if "select" in state.steps_completed:
        # Must have retrieved first
        if "retrieve" not in state.steps_completed:
            errors.append("❌ CRITICAL: Selection occurred before retrieval!")
        
        if len(state.selected_chunks) == 0:
            errors.append("Selection step completed but no chunks selected")
    
    # Validate generation
    if "generate" in state.steps_completed:
        # Must have retrieved AND selected first
        if "retrieve" not in state.steps_completed:
            errors.append("❌ CRITICAL: Generation occurred before retrieval!")
        if "select" not in state.steps_completed:
            errors.append("❌ CRITICAL: Generation occurred before selection!")
        
        if state.final_answer is None or state.final_answer.strip() == "":
            errors.append("Generation step completed but final_answer is empty")
        
        # Check if answer references retrieved content
        if state.final_answer and len(state.selected_chunks) > 0:
            # Very simple check: does answer mention any doc titles?
            doc_titles = [d.title for d in state.retrieved_documents]
            has_citation = any(title.lower() in state.final_answer.lower() 
                              for title in doc_titles)
            if not has_citation:
                errors.append("⚠️  WARNING: Answer may not cite retrieved sources")
    
    if errors:
        print("🔴 RAG STATE VALIDATION FAILED")
        for error in errors:
            print(f"  {error}")
        return False
    else:
        print("✅ RAG state validation passed")
        return True


# Test validation on correct workflow
print("=" * 70)
print("  VALIDATING CORRECT RAG WORKFLOW")
print("=" * 70)
validate_rag_state(rag_state)

# Test validation on incorrect workflow
print("\n" + "=" * 70)
print("  VALIDATING INCORRECT RAG WORKFLOW")
print("=" * 70)

bad_rag_state = RAGAgentState(query="What is AI?")
bad_rag_state.steps_completed = ["generate"]  # Skipped retrieval!
bad_rag_state.final_answer = "AI is artificial intelligence."

validate_rag_state(bad_rag_state)
print("=" * 70)

  VALIDATING CORRECT RAG WORKFLOW
✅ RAG state validation passed

  VALIDATING INCORRECT RAG WORKFLOW
🔴 RAG STATE VALIDATION FAILED
  ❌ CRITICAL: Generation occurred before retrieval!
  ❌ CRITICAL: Generation occurred before selection!


## 6.6 Why Retrieval Outputs Belong in State: Summary

### Traceability
- We can inspect **exactly what was retrieved**
- We can verify **which chunks were selected**
- We can trace **how documents → answer**

### Debuggability
- If the answer is wrong, check `retrieved_documents`
- If retrieval is bad, check `query`
- If selection is too aggressive, check `selected_chunks`

### Validation
- Ensure retrieval happened before generation
- Verify sufficient context was retrieved
- Check that answer cites sources

### Auditability
- Log complete state for compliance
- Recreate agent decisions from state
- Prove which sources were used

---

## 🎯 Key Takeaway

> **RAG agents must store retrieval outputs in state. Without this, the agent becomes a black box — you cannot debug, validate, or audit its reasoning process.**

---

---

# Progress Checkpoint: Sections 4-6 Complete

## What We've Implemented

| Section | Topic | Key Concepts |
|---------|-------|--------------|
| **Section 4** | Single-Agent Multi-Step State | State evolution, field rationale, snapshots |
| **Section 5** | State Validation | Validation functions, failure detection, best practices |
| **Section 6** | RAG Agent State | Retrieval storage, step ordering, traceability |

---

## Coming Next

The remaining sections will cover:

- **Section 7:** Visual diagrams of state flow
- **Section 8:** Multi-agent systems with shared state
- **Section 9:** Common state design mistakes
- **Section 10:** Final mental model and checklist

---

<a id="section-7"></a>
# Section 7: Visual Diagrams of State Flow

Diagrams help us understand how state flows through agent systems. In this section, we'll visualize state evolution using ASCII diagrams.

---

## 7.1 Single-Agent Step Flow

This diagram shows how a single agent processes state through sequential steps.

In [24]:
print("""
┌─────────────────────────────────────────────────────────────┐
│              SINGLE-AGENT STATE FLOW                        │
└─────────────────────────────────────────────────────────────┘

   Initial State
   ┌──────────────────┐
   │ goal: "..."      │
   │ steps: []        │
   │ results: {}      │
   │ output: None     │
   └────────┬─────────┘
            │
            ▼
   ┌────────────────────┐
   │   STEP 1: Plan     │
   │  (Reads State)     │
   └────────┬───────────┘
            │ Modifies State
            ▼
   ┌──────────────────┐
   │ goal: "..."      │
   │ steps: ["plan"]  │
   │ results: {}      │
   │ output: None     │
   └────────┬─────────┘
            │
            ▼
   ┌────────────────────┐
   │  STEP 2: Execute   │
   │  (Reads State)     │
   └────────┬───────────┘
            │ Modifies State
            ▼
   ┌────────────────────────┐
   │ goal: "..."            │
   │ steps: ["plan","exec"] │
   │ results: {...}         │
   │ output: None           │
   └────────┬───────────────┘
            │
            ▼
   ┌──────────────────────┐
   │ STEP 3: Synthesize   │
   │  (Reads State)       │
   └────────┬─────────────┘
            │ Modifies State
            ▼
   ┌──────────────────────────────┐
   │ goal: "..."                  │
   │ steps: ["plan","exec","syn"] │
   │ results: {...}               │
   │ output: "Final result!"      │
   └──────────────────────────────┘
            │
            ▼
      Final State
      (Complete!)

KEY OBSERVATIONS:
• State flows DOWN through sequential steps
• Each step READS the current state
• Each step MODIFIES the state
• State accumulates information (never loses data)
• Final state contains complete execution history
""")


┌─────────────────────────────────────────────────────────────┐
│              SINGLE-AGENT STATE FLOW                        │
└─────────────────────────────────────────────────────────────┘

   Initial State
   ┌──────────────────┐
   │ goal: "..."      │
   │ steps: []        │
   │ results: {}      │
   │ output: None     │
   └────────┬─────────┘
            │
            ▼
   ┌────────────────────┐
   │   STEP 1: Plan     │
   │  (Reads State)     │
   └────────┬───────────┘
            │ Modifies State
            ▼
   ┌──────────────────┐
   │ goal: "..."      │
   │ steps: ["plan"]  │
   │ results: {}      │
   │ output: None     │
   └────────┬─────────┘
            │
            ▼
   ┌────────────────────┐
   │  STEP 2: Execute   │
   │  (Reads State)     │
   └────────┬───────────┘
            │ Modifies State
            ▼
   ┌────────────────────────┐
   │ goal: "..."            │
   │ steps: ["plan","exec"] │
   │ results: {...}         │
   │ output: None           │
 

## 7.2 State Evolution Across Steps

This diagram shows how specific fields change as the agent progresses.

In [25]:
print("""
STATE FIELD EVOLUTION TIMELINE
═══════════════════════════════════════════════════════════════

Time →   T0 (Init)    T1 (Plan)    T2 (Execute)   T3 (Synthesize)
─────────────────────────────────────────────────────────────────
goal     "Task X"     "Task X"     "Task X"       "Task X"
         ✓ Set       ✓ Same       ✓ Same         ✓ Same

steps    []          ["plan"]     ["plan",       ["plan","exec",
                                   "exec"]        "syn"]
         Empty       Added 1      Added 1        Added 1

subtasks []          ["A","B"]    ["A","B"]      ["A","B"]
         Empty       ✓ Created    ✓ Same         ✓ Same

results  {}          {}           {A: "...",     {A: "...",
                                   B: "..."}     B: "..."}
         Empty       Empty        ✓ Populated    ✓ Same

output   None        None         None           "Final!"
         Not ready   Not ready    Not ready      ✓ Generated
─────────────────────────────────────────────────────────────────

PATTERN: Fields populate LEFT-TO-RIGHT as agent progresses
         Once set, fields remain available to future steps
         State grows monotonically (accumulates data)
""")


STATE FIELD EVOLUTION TIMELINE
═══════════════════════════════════════════════════════════════

Time →   T0 (Init)    T1 (Plan)    T2 (Execute)   T3 (Synthesize)
─────────────────────────────────────────────────────────────────
goal     "Task X"     "Task X"     "Task X"       "Task X"
         ✓ Set       ✓ Same       ✓ Same         ✓ Same

steps    []          ["plan"]     ["plan",       ["plan","exec",
                                   "exec"]        "syn"]
         Empty       Added 1      Added 1        Added 1

subtasks []          ["A","B"]    ["A","B"]      ["A","B"]
         Empty       ✓ Created    ✓ Same         ✓ Same

results  {}          {}           {A: "...",     {A: "...",
                                   B: "..."}     B: "..."}
         Empty       Empty        ✓ Populated    ✓ Same

output   None        None         None           "Final!"
         Not ready   Not ready    Not ready      ✓ Generated
────────────────────────────────────────────────────────────────

## 7.3 The Four Layers: Data → Knowledge → Logic → Orchestration

This diagram shows how state relates to the broader agent architecture.

In [27]:
print("""
┌───────────────────────────────────────────────────────────┐
│                 AGENTIC ARCHITECTURE LAYERS               │
└───────────────────────────────────────────────────────────┘

    Layer 4: ORCHESTRATION
    ┌─────────────────────────────────────┐
    │  Workflow Engine / Graph Execution  │
    │  • When to call which step          │
    │  • Routing logic                    │
    │  • Conditional branching            │
    └──────────────┬──────────────────────┘
                   │ Uses
                   ▼
    Layer 3: LOGIC (Agent Functions)
    ┌─────────────────────────────────────┐
    │  step1_plan()                       │
    │  step2_execute()                    │
    │  step3_synthesize()                 │
    │  • Business logic                   │
    │  • Decision making                  │
    └──────────────┬──────────────────────┘
                   │ Reads/Writes
                   ▼
    Layer 2: KNOWLEDGE (State)  ⭐ THIS IS WHERE STATE LIVES
    ┌─────────────────────────────────────┐
    │  ┌───────────────────────────────┐  │
    │  │ goal: "..."                   │  │
    │  │ steps_completed: [...]        │  │
    │  │ intermediate_results: {...}   │  │
    │  │ final_output: "..."           │  │
    │  └───────────────────────────────┘  │
    │  • Structured memory                │
    │  • What the agent "knows"           │
    └──────────────┬──────────────────────┘
                   │ Contains
                   ▼
    Layer 1: DATA (Raw Information)
    ┌─────────────────────────────────────┐
    │  • Tool outputs                     │
    │  • LLM responses                    │
    │  • Retrieved documents              │
    │  • User inputs                      │
    │  • External API results             │
    └─────────────────────────────────────┘

KEY INSIGHT:
State (Layer 2) is the BRIDGE between:
  • Raw data (Layer 1) ←→ Business logic (Layer 3)
  
Without state, logic cannot access data, and orchestration
cannot track progress!
""")


┌───────────────────────────────────────────────────────────┐
│                 AGENTIC ARCHITECTURE LAYERS               │
└───────────────────────────────────────────────────────────┘

    Layer 4: ORCHESTRATION
    ┌─────────────────────────────────────┐
    │  Workflow Engine / Graph Execution  │
    │  • When to call which step          │
    │  • Routing logic                    │
    │  • Conditional branching            │
    └──────────────┬──────────────────────┘
                   │ Uses
                   ▼
    Layer 3: LOGIC (Agent Functions)
    ┌─────────────────────────────────────┐
    │  step1_plan()                       │
    │  step2_execute()                    │
    │  step3_synthesize()                 │
    │  • Business logic                   │
    │  • Decision making                  │
    └──────────────┬──────────────────────┘
                   │ Reads/Writes
                   ▼
    Layer 2: KNOWLEDGE (State)  ⭐ THIS IS WHERE STATE LIVES
    ┌─────────

## 7.4 Single-Agent vs Multi-Agent State Flow

This diagram compares state management in single-agent and multi-agent systems.

In [29]:
print("""
SINGLE-AGENT STATE                 MULTI-AGENT STATE
═════════════════                  ══════════════════

    ┌────────┐                         ┌──────────────────┐
    │ STATE  │                         │  SHARED STATE    │
    │        │                         │  (Global View)   │
    │ • goal │                         │                  │
    │ • data │                         │ • overall_goal   │
    │ • out  │                         │ • agent_outputs  │
    └───┬────┘                         │ • coordination   │
        │                              └────┬────┬────────┘
        │ Read/Write                        │    │
        │                                   │    │
   ┌────▼────┐                         ┌────▼─┐ ┌▼─────┐
   │  Agent  │                         │Agent │ │Agent │
   │         │                         │  A   │ │  B   │
   │ Step 1  │                         │      │ │      │
   │ Step 2  │                         │Local │ │Local │
   │ Step 3  │                         │State │ │State │
   └─────────┘                         └──┬───┘ └───┬──┘
                                          │         │
   Simple!                                │ Writes  │ Writes
   All steps share                        │         │
   same state                             ▼         ▼
                                      ┌──────────────────┐
                                      │  SHARED STATE    │
                                      │  (Updated)       │
                                      └──────────────────┘
                                      
                                      Complex!
                                      Need state contracts
                                      and coordination

SINGLE-AGENT:                      MULTI-AGENT:
✓ One state object                 ✓ Shared global state
✓ Linear state flow                ✓ Local agent states
✓ Simple to reason about           ✓ State handoff/merge
✗ No parallelism                   ✓ Parallel execution
✗ No specialization                ✓ Agent specialization
                                   ✗ Coordination complexity
                                   ✗ Race conditions possible
""")


SINGLE-AGENT STATE                 MULTI-AGENT STATE
═════════════════                  ══════════════════

    ┌────────┐                         ┌──────────────────┐
    │ STATE  │                         │  SHARED STATE    │
    │        │                         │  (Global View)   │
    │ • goal │                         │                  │
    │ • data │                         │ • overall_goal   │
    │ • out  │                         │ • agent_outputs  │
    └───┬────┘                         │ • coordination   │
        │                              └────┬────┬────────┘
        │ Read/Write                        │    │
        │                                   │    │
   ┌────▼────┐                         ┌────▼─┐ ┌▼─────┐
   │  Agent  │                         │Agent │ │Agent │
   │         │                         │  A   │ │  B   │
   │ Step 1  │                         │      │ │      │
   │ Step 2  │                         │Local │ │Local │
   │ Step 3  │          

---

## 🎯 Key Takeaway: Section 7

> **Visual diagrams reveal patterns that code alone cannot show. State flows sequentially in single agents, accumulates monotonically, and serves as the bridge between data and logic. In multi-agent systems, state coordination becomes the critical challenge.**

---

<a id="section-8"></a>
# Section 8: From Single Agent to Multi-Agent System (MAS)

Now we scale from a single agent to **multiple collaborating agents**, each with their own specialty.

We'll build a simple MAS with:
- **Agent A (Researcher):** Gathers information
- **Agent B (Writer):** Synthesizes into a report

---

## 8.1 The State Challenge in MAS

### Problem: How do agents share information?

**Option 1: Shared Global State** ← We'll use this
- All agents read/write to one state object
- Simple but requires coordination

**Option 2: Message Passing**
- Agents send messages to each other
- Complex but more isolated

**Option 3: Hybrid**
- Shared state + explicit handoffs
- Production systems often use this

---

## 8.2 Designing the MAS State Schema

In [ ]:
@dataclass
class MultiAgentState:
    """
    Shared state for a multi-agent system.
    
    Design principles:
    1. Track which agent did what
    2. Store outputs from each agent
    3. Coordinate handoffs between agents
    """
    # INPUT: The overall task
    task: str
    
    # COORDINATION: Which agents have completed their work
    agents_completed: list[str] = field(default_factory=list)
    
    # AGENT A OUTPUT: Research findings
    research_data: Optional[dict] = None
    
    # AGENT B OUTPUT: Written report
    final_report: Optional[str] = None
    
    # META: Execution log
    execution_log: list[str] = field(default_factory=list)


print("✅ Multi-Agent State Schema Defined")
print()
print("State Fields:")
print("  • task               → Overall goal")
print("  • agents_completed   → Coordination tracking")
print("  • research_data      → Agent A's output")
print("  • final_report       → Agent B's output")
print("  • execution_log      → Audit trail")

## 8.3 Implementing the Agents

In [ ]:
def agent_a_researcher(state: MultiAgentState) -> MultiAgentState:
    """
    Agent A: Researcher
    
    Responsibilities:
    - Gather data based on the task
    - Store findings in shared state
    - Signal completion
    """
    print("🔬 AGENT A (Researcher) Starting...")
    print("-" * 50)
    
    # Simulate research (would use tools/APIs in production)
    research_findings = {
        "topic": state.task,
        "key_facts": [
            "Transformers revolutionized NLP",
            "Attention mechanism is the core innovation",
            "BERT and GPT are popular variants"
        ],
        "sources": ["Paper 1", "Paper 2", "Paper 3"],
        "confidence": 0.95
    }
    
    # Write to shared state
    state.research_data = research_findings
    state.agents_completed.append("researcher")
    state.execution_log.append("Agent A completed research")
    
    print(f"Task: {state.task}")
    print(f"Found {len(research_findings['key_facts'])} key facts")
    print(f"Confidence: {research_findings['confidence']}")
    print("✅ Research complete, data stored in shared state")
    print("-" * 50)
    
    return state


def agent_b_writer(state: MultiAgentState) -> MultiAgentState:
    """
    Agent B: Writer
    
    Responsibilities:
    - Read research data from shared state
    - Synthesize into a report
    - Store final output
    """
    print("\n✍️  AGENT B (Writer) Starting...")
    print("-" * 50)
    
    # Validate: Can only run after researcher
    if "researcher" not in state.agents_completed:
        raise ValueError("❌ Writer cannot run before Researcher!")
    
    # Read from shared state
    research = state.research_data
    
    if research is None:
        raise ValueError("❌ No research data available!")
    
    # Synthesize report
    report_lines = [
        f"# Report: {research['topic']}",
        "",
        "## Executive Summary",
        f"This report covers {research['topic']} based on {len(research['sources'])} sources.",
        "",
        "## Key Findings",
    ]
    
    for i, fact in enumerate(research['key_facts'], 1):
        report_lines.append(f"{i}. {fact}")
    
    report_lines.extend([
        "",
        "## Sources",
        ", ".join(research['sources']),
        "",
        f"**Confidence Level:** {research['confidence'] * 100}%"
    ])
    
    state.final_report = "\n".join(report_lines)
    state.agents_completed.append("writer")
    state.execution_log.append("Agent B completed writing")
    
    print(f"Synthesized report from {len(research['key_facts'])} facts")
    print("✅ Report complete, stored in shared state")
    print("-" * 50)
    
    return state


print("✅ Multi-Agent functions defined:")
print("   - agent_a_researcher()")
print("   - agent_b_writer()")

## 8.4 Running the Multi-Agent System

In [ ]:
# Initialize shared state
mas_state = MultiAgentState(task="Summarize transformer architectures")

print("=" * 70)
print("  MULTI-AGENT SYSTEM EXECUTION")
print("=" * 70)
print()

print("📸 Initial Shared State:")
display_state(mas_state, "MAS Initial State")

# Agent A executes
mas_state = agent_a_researcher(mas_state)
print("\n📸 State After Agent A:")
display_state(mas_state, "State After Researcher")

# Agent B executes (depends on Agent A)
mas_state = agent_b_writer(mas_state)
print("\n📸 Final Shared State:")
display_state(mas_state, "Final MAS State")

# Display final report
print("=" * 70)
print("  FINAL REPORT")
print("=" * 70)
print(mas_state.final_report)
print("=" * 70)

## 8.5 Validating MAS State Integrity

In [ ]:
def validate_mas_state(state: MultiAgentState) -> bool:
    """
    Validate multi-agent state with coordination checks.
    
    Critical invariants:
    1. Agent B cannot run before Agent A
    2. Each agent's output must be present after completion
    3. Execution log must match agents_completed
    """
    errors = []
    
    # Check basic fields
    if not state.task:
        errors.append("Task is missing")
    
    # Validate Agent A completion
    if "researcher" in state.agents_completed:
        if state.research_data is None:
            errors.append("Researcher marked complete but research_data is None")
    
    # Validate Agent B completion
    if "writer" in state.agents_completed:
        # Must have researcher first
        if "researcher" not in state.agents_completed:
            errors.append("❌ CRITICAL: Writer completed before Researcher!")
        
        if state.final_report is None:
            errors.append("Writer marked complete but final_report is None")
        
        # Validate report references research
        if state.research_data and state.final_report:
            if state.task not in state.final_report:
                errors.append("⚠️  Report doesn't mention the task")
    
    # Validate execution log consistency
    if len(state.execution_log) != len(state.agents_completed):
        errors.append(f"Execution log has {len(state.execution_log)} entries but {len(state.agents_completed)} agents completed")
    
    if errors:
        print("🔴 MAS STATE VALIDATION FAILED")
        for error in errors:
            print(f"  {error}")
        return False
    else:
        print("✅ MAS state validation passed")
        return True


# Test correct execution
print("=" * 70)
print("  VALIDATING CORRECT MAS EXECUTION")
print("=" * 70)
validate_mas_state(mas_state)

# Test incorrect execution (Agent B before Agent A)
print("\n" + "=" * 70)
print("  VALIDATING INCORRECT MAS EXECUTION")
print("=" * 70)

bad_mas_state = MultiAgentState(task="Test")
bad_mas_state.agents_completed = ["writer"]  # Wrong order!
bad_mas_state.final_report = "Some report"

validate_mas_state(bad_mas_state)
print("=" * 70)

## 8.6 Local vs Shared State

### Why This Matters

| Aspect | Local State | Shared State |
|--------|-------------|--------------|
| **Scope** | Private to one agent | Visible to all agents |
| **Purpose** | Temporary calculations | Cross-agent coordination |
| **Lifetime** | Duration of agent execution | Entire workflow |
| **Example** | Intermediate LLM responses | Final outputs, coordination flags |

### When to Use Each

**Use Local State:**
- Agent-specific temporary data
- Private reasoning traces
- Optimization/caching

**Use Shared State:**
- Outputs consumed by other agents
- Coordination signals
- Final deliverables

---

## 🎯 Key Takeaway: Section 8

> **Multi-agent systems require explicit state contracts. Agents must know what they can read, what they must write, and in what order. Without clear state coordination, MAS becomes unpredictable and buggy.**

---

<a id="section-9"></a>
# Section 9: Common State Design Mistakes

Learning from failures is critical. Here are the most common state design mistakes and why they break agents.

---

## 9.1 Mistake #1: Overloaded State

**What it looks like:**

In [ ]:
# MISTAKE #1: Overloaded State
# Putting EVERYTHING in one giant state object

@dataclass
class OverloadedState:
    """BAD: Too much unrelated data in one state."""
    # User inputs
    user_query: str = ""
    user_preferences: dict = field(default_factory=dict)
    
    # Search results
    search_results: list = field(default_factory=list)
    
    # Retrieved docs
    documents: list = field(default_factory=list)
    
    # LLM generations
    llm_response_1: str = ""
    llm_response_2: str = ""
    llm_response_3: str = ""
    
    # Tool outputs
    tool_output_a: dict = field(default_factory=dict)
    tool_output_b: dict = field(default_factory=dict)
    
    # Random stuff
    debug_info: str = ""
    temp_cache: dict = field(default_factory=dict)
    random_flags: list = field(default_factory=list)
    
    # ... and 20 more fields


print("❌ WHY THIS IS BAD:")
print("-" * 60)
print("1. Impossible to understand what the agent is doing")
print("2. Hard to validate (which fields are required when?)")
print("3. Difficult to debug (too much noise)")
print("4. Poor maintainability (everything is entangled)")
print("5. No clear workflow (fields don't map to steps)")
print()
print("✅ BETTER APPROACH:")
print("   Create FOCUSED state schemas for specific workflows")
print("   Only include fields that serve the agent's purpose")

## 9.2 Mistake #2: Under-Specified State

**What it looks like:**

In [ ]:
# MISTAKE #2: Under-Specified State
# Not enough information to reconstruct what happened

@dataclass
class UnderSpecifiedState:
    """BAD: Missing critical tracking fields."""
    result: str = ""  # What result? From what step?
    data: dict = field(default_factory=dict)  # What kind of data?


# Try to use this state
state = UnderSpecifiedState(
    result="Success",
    data={"value": 42}
)

print("❌ WHY THIS IS BAD:")
print("-" * 60)
print("1. Can't tell what steps have been completed")
print("2. Can't debug failures (no execution trace)")
print("3. Can't validate correctness (no invariants)")
print("4. Can't reconstruct agent reasoning")
print("5. 'result' and 'data' are too vague")
print()
print("Example problems:")
print(f"  • What does result='{state.result}' mean?")
print(f"  • What step produced this data: {state.data}?")
print(f"  • Did retrieval happen? Unknown.")
print(f"  • Can we proceed to the next step? Unknown.")
print()
print("✅ BETTER APPROACH:")
print("   Include explicit fields for:")
print("   • What steps completed (steps_completed)")
print("   • What was retrieved (retrieved_docs)")
print("   • What decisions were made (selections)")
print("   • Timestamps, errors, metadata")

## 9.3 Mistake #3: Reusing SimpleState Incorrectly

**What it looks like:**

In [ ]:
# MISTAKE #3: Reusing SimpleState for Complex Workflows
# Trying to use message-only state for multi-step agents

# Attempt to build RAG with SimpleState
simple_rag_state: SimpleAgentState = {"messages": []}

# Step 1: User asks question
simple_rag_state["messages"].append({
    "role": "user",
    "content": "What are transformers?"
})

# Step 2: Try to store search results in messages (WRONG!)
search_results = ["Doc1", "Doc2", "Doc3"]
simple_rag_state["messages"].append({
    "role": "system",
    "content": f"Search results: {search_results}"  # Unstructured!
})

# Step 3: Try to access search results later
# How do we parse this back out???
last_message = simple_rag_state["messages"][-1]["content"]
# Need to parse string? Error-prone!

print("❌ WHY THIS IS BAD:")
print("-" * 60)
print("1. Search results buried in unstructured text")
print("2. No type safety (everything is a string)")
print("3. Parsing messages is fragile and error-prone")
print("4. Can't validate that retrieval happened")
print("5. Difficult to serialize/deserialize correctly")
print()
print("Example problems:")
print(f"  • Last message: {last_message}")
print(f"  • How do we extract the list of docs?")
print(f"  • What if the format changes?")
print(f"  • How do we validate this state?")
print()
print("✅ BETTER APPROACH:")
print("   Use structured state with explicit fields:")
print("   • query: str")
print("   • retrieved_docs: list[Document]")
print("   • selected_chunks: list[str]")
print("   • final_answer: str")

## 9.4 Mistake #4: Hidden State Inside Prompts

**What it looks like:**

In [ ]:
# MISTAKE #4: Hidden State Inside Prompts
# Encoding state in prompt text instead of structured fields

@dataclass
class HiddenStateExample:
    """State schema looks simple..."""
    query: str = ""
    answer: str = ""
    # But where are the retrieved docs? Hidden in the prompt!


# The "state" is actually hidden in the prompt
prompt_with_hidden_state = """
You are a helpful assistant.

Previously, you searched for documents and found:
1. Document A: "Transformers use attention..."
2. Document B: "BERT is bidirectional..."
3. Document C: "GPT is autoregressive..."

User selected documents: A and B

Now answer the question: What are transformers?
"""

state = HiddenStateExample(
    query="What are transformers?",
    answer=""  # To be filled
)

print("❌ WHY THIS IS BAD:")
print("-" * 60)
print("1. State is INVISIBLE in the state object")
print("2. Cannot validate retrieval happened")
print("3. Cannot inspect what was retrieved")
print("4. Prompt and state are out of sync")
print("5. Cannot serialize/restore state correctly")
print("6. Debugging requires reading prompts, not state")
print()
print("Example problems:")
print(f"  • State says query='{state.query}'")
print(f"  • But docs are hidden in prompt text!")
print(f"  • How do we validate doc selection?")
print(f"  • How do we log what was retrieved?")
print(f"  • If we reload state, docs are LOST")
print()
print("✅ BETTER APPROACH:")
print("   Store ALL relevant data in state:")
print("   • retrieved_docs: list[Document]")
print("   • selected_docs: list[str]")
print("   • Build prompts FROM state, not instead of state")

## 9.5 Summary of Common Mistakes

| Mistake | Symptom | Fix |
|---------|---------|-----|
| **Overloaded State** | Too many unrelated fields | Create focused, purpose-specific schemas |
| **Under-Specified State** | Missing tracking/metadata | Add steps_completed, timestamps, errors |
| **SimpleState Misuse** | Structured data in messages | Use explicit typed fields |
| **Hidden State in Prompts** | State invisible to code | Store everything in state, build prompts from it |

---

## 🎯 Key Takeaway: Section 9

> **State design mistakes create technical debt that compounds over time. A poorly designed state schema makes agents unpredictable, undebuggable, and unmaintainable. Get the state right from the start.**

---

<a id="section-10"></a>
# Section 10: Final Mental Model & Checklist

We've journeyed from first principles to production-ready concepts. Let's consolidate everything into a clear mental model and actionable checklist.

---

## 10.1 The Complete Mental Model of Agent State

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           THE COMPLETE MENTAL MODEL OF AGENT STATE          ║
╚══════════════════════════════════════════════════════════════╝

CORE PRINCIPLES
═══════════════

1. STATE IS THE AGENT'S WORKING MEMORY
   • Not the schema (blueprint)
   • Not the messages (conversation)
   • Not the code (logic)
   → The RUNTIME DATA the agent carries between steps

2. STATE DEFINES THE REASONING BOUNDARY
   • Agent can only reason about what's IN state
   • Anything outside state is INVISIBLE
   • State determines what's possible

3. STATE MUST BE EXPLICIT AND OBSERVABLE
   • Inspectable at any time
   • Serializable for logging
   • Validatable at each step

4. STATE ACCUMULATES MONOTONICALLY
   • Data flows IN, never thrown away
   • Each step adds information
   • History preserved for debugging


THE STATE LIFECYCLE
═══════════════════

   Design → Initialize → Execute → Validate → Serialize
      ↑                                            ↓
      └────────────── Iterate ─────────────────────┘


STATE ARCHITECTURE
══════════════════

   Layer 4: ORCHESTRATION (workflow control)
               ↓
   Layer 3: LOGIC (agent functions)
               ↓
   Layer 2: KNOWLEDGE (STATE) ⭐ ← You are here
               ↓
   Layer 1: DATA (raw inputs/outputs)


SINGLE-AGENT vs MULTI-AGENT
════════════════════════════

   Single-Agent:              Multi-Agent:
   • One state object         • Shared global state
   • Sequential steps         • + Local agent states
   • Simple reasoning         • + Coordination logic


VALIDATION IS MANDATORY
════════════════════════

   After each step:
   1. Check required fields exist
   2. Verify correct data types
   3. Assert step ordering
   4. Validate invariants
   5. Log any failures
""")


## 10.2 The Reusable State Design Checklist

Use this checklist for EVERY agent you build.

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║          AGENT STATE DESIGN CHECKLIST (v1.0)                 ║
╚══════════════════════════════════════════════════════════════╝

BEFORE WRITING ANY CODE
════════════════════════

☐ 1. UNDERSTAND THE TASK
      What is the agent supposed to accomplish?
      
☐ 2. ENUMERATE STEPS
      What are the distinct phases/steps?
      Example: Plan → Retrieve → Analyze → Generate
      
☐ 3. IDENTIFY STATE FIELDS
      For each step, ask: "What must be remembered?"
      

DESIGNING THE SCHEMA
════════════════════

☐ 4. DEFINE INPUTS
      ☐ User query/goal
      ☐ Configuration parameters
      ☐ Initial context
      
☐ 5. DEFINE TRACKING FIELDS
      ☐ steps_completed: list[str]
      ☐ current_step: str (optional)
      ☐ errors: list (if needed)
      
☐ 6. DEFINE INTERMEDIATE RESULTS
      ☐ One field per major step output
      ☐ Retrieved documents
      ☐ Analysis results
      ☐ Draft outputs
      
☐ 7. DEFINE FINAL OUTPUTS
      ☐ final_answer / final_report
      ☐ Citations / sources
      ☐ Confidence scores
      
☐ 8. DEFINE METADATA
      ☐ created_at timestamp
      ☐ execution_log (for debugging)
      ☐ agent_id (for MAS)


VALIDATION DESIGN
═════════════════

☐ 9. CREATE VALIDATION RULES
      For each step:
      ☐ Required fields at this step?
      ☐ Allowed transitions from previous step?
      ☐ Data type checks
      ☐ Business logic invariants
      
☐ 10. IMPLEMENT VALIDATION FUNCTIONS
       ☐ validate_<agent>_state(state, step) -> bool
       ☐ Clear error messages
       ☐ Test failure cases


IMPLEMENTATION CHECKS
════════════════════

☐ 11. USE PROPER TYPING
       ☐ TypedDict, dataclass, or Pydantic
       ☐ Optional[T] for nullable fields
       ☐ list[T] not list
       ☐ dict[K, V] not dict
       
☐ 12. PROVIDE DEFAULTS
       ☐ field(default_factory=list) for lists
       ☐ field(default_factory=dict) for dicts
       ☐ None for optional fields
       
☐ 13. ADD HELPER METHODS (if using dataclass)
       ☐ is_complete() -> bool
       ☐ current_step() -> str
       ☐ to_dict() if needed


PRODUCTION READINESS
═══════════════════

☐ 14. SERIALIZATION
       ☐ State can be converted to JSON
       ☐ State can be restored from JSON
       ☐ No circular references
       
☐ 15. LOGGING & DEBUGGING
       ☐ State logged at each step
       ☐ State snapshots saved
       ☐ Execution trace preserved
       
☐ 16. DOCUMENTATION
       ☐ Each field has a docstring
       ☐ State schema documented
       ☐ Example state provided


MULTI-AGENT SYSTEMS (if applicable)
═══════════════════════════════════

☐ 17. COORDINATION FIELDS
       ☐ agents_completed: list[str]
       ☐ agent_outputs: dict[agent_id, output]
       
☐ 18. STATE CONTRACTS
       ☐ Document what each agent reads
       ☐ Document what each agent writes
       ☐ Document execution order constraints
       
☐ 19. HANDOFF VALIDATION
       ☐ Validate Agent B cannot run before Agent A
       ☐ Validate data exists before handoff
       ☐ Clear error messages for violations


FINAL CHECKS
════════════

☐ 20. TEST COVERAGE
       ☐ Test with valid state
       ☐ Test with invalid state
       ☐ Test step ordering violations
       ☐ Test missing required fields
       
☐ 21. REVIEW AGAINST COMMON MISTAKES
       ☐ Not overloaded?
       ☐ Not under-specified?
       ☐ Not misusing SimpleState?
       ☐ Not hiding state in prompts?

═══════════════════════════════════════════════════════════════

✅ If all boxes are checked, your state design is production-ready!
""")

## 10.3 Scaling State to Production Systems

### From Prototype to Production

| Aspect | Prototype | Production |
|--------|-----------|------------|
| **State Storage** | In-memory dict | Database / Redis |
| **Validation** | Optional | Required at every step |
| **Logging** | Print statements | Structured logging + tracing |
| **Serialization** | JSON.dumps() | Versioned schemas + migration |
| **Error Handling** | Exceptions | Graceful degradation + recovery |
| **Concurrency** | Single-threaded | Locks / transactions |

### Production Considerations

**1. State Persistence**
- Store state in database for durability
- Use Redis for fast state access
- Implement state checkpointing

**2. State Versioning**
- Version your state schemas
- Implement migration logic
- Handle backward compatibility

**3. State Observability**
- Log state transitions
- Create state dashboards
- Alert on validation failures

**4. State Recovery**
- Implement idempotent operations
- Support state rollback
- Enable replay from checkpoints

**5. State Security**
- Encrypt sensitive fields
- Implement access controls
- Audit state access

---

## 10.4 Final Wisdom

### The Three Laws of Agent State

1. **State is Truth**: The state IS the source of truth. Not prompts, not messages, not logs.

2. **Design Before Code**: Never implement an agent without first designing its state schema.

3. **Validate Always**: Every state transition must be validated. Silent failures are unacceptable.

### When to Rethink Your State Design

- ❌ You can't explain what a field does
- ❌ Validation logic is complex and fragile
- ❌ You need to parse strings to extract data
- ❌ Debugging requires reading prompts
- ❌ State doesn't match your agent's workflow

### Remember

> **Good state design makes agents predictable, debuggable, and maintainable. Poor state design makes agents unpredictable, opaque, and fragile. The time you invest in state design pays exponential dividends in production.**

---

---

# 🎓 Conclusion

## What You've Learned

You now have a complete understanding of agent state from first principles:

1. ✅ **Conceptual Foundation**: What state is (and isn't)
2. ✅ **Design Process**: How to design state before code
3. ✅ **Implementation**: Single-agent and multi-agent systems
4. ✅ **Validation**: How to ensure state correctness
5. ✅ **Best Practices**: Common mistakes and how to avoid them
6. ✅ **Production**: Scaling state to real systems

---

## Next Steps

1. **Practice**: Implement your own agent with proper state design
2. **Experiment**: Try different state schemas for the same task
3. **Validate**: Add validation to existing agents
4. **Refactor**: Review old code and improve state design
5. **Scale**: Apply these principles to multi-agent systems

---

## Keep This Checklist

Every time you build an agent, return to **Section 10.2's checklist**. It will save you hours of debugging and refactoring.

---

## Thank You!

You've completed the journey from "What is agent state?" to production-ready state design. Use this knowledge wisely!

**Happy building! 🚀**

---